# Parse the DrugBank XML and extract TSVs

Run using Python 3 to avoid a non-ascii character error when writing to file with the csv module.

In [1]:
import os
import csv
import gzip
import collections
import re
import io
import json
import xml.etree.ElementTree as ET

import requests
import pandas
import numpy as np

In [2]:
import gseapy as gp
from gseapy.parser import Biomart

In [3]:
bm = Biomart(verbose=False, host="asia.ensembl.org")

In [4]:
xml_path = os.path.join('./Primary_target_databases/DrugBank_2022/', 'full database.xml')
with open(xml_path) as xml_file:
    tree = ET.parse(xml_file)
root = tree.getroot()

In [5]:
ns = '{http://www.drugbank.ca}'
inchikey_template = "{ns}calculated-properties/{ns}property[{ns}kind='InChIKey']/{ns}value"
inchi_template = "{ns}calculated-properties/{ns}property[{ns}kind='InChI']/{ns}value"

rows = list()
for i, drug in enumerate(root):
    row = collections.OrderedDict()
    assert drug.tag == ns + 'drug'
    row['type'] = drug.get('type')
    row['drugbank_id'] = drug.findtext(ns + "drugbank-id[@primary='true']")
    row['name'] = drug.findtext(ns + "name")
    row['description'] = drug.findtext(ns + "description")
    row['groups'] = [group.text for group in
        drug.findall("{ns}groups/{ns}group".format(ns = ns))]
    row['atc_codes'] = [code.get('code') for code in
        drug.findall("{ns}atc-codes/{ns}atc-code".format(ns = ns))]
    row['categories'] = [x.findtext(ns + 'category') for x in
        drug.findall("{ns}categories/{ns}category".format(ns = ns))]
    row['inchi'] = drug.findtext(inchi_template.format(ns = ns))
    row['inchikey'] = drug.findtext(inchikey_template.format(ns = ns))
    
    # Add drug aliases
    aliases = {
        elem.text for elem in 
        drug.findall("{ns}international-brands/{ns}international-brand".format(ns = ns)) +
        drug.findall("{ns}synonyms/{ns}synonym[@language='English']".format(ns = ns)) +
        drug.findall("{ns}international-brands/{ns}international-brand".format(ns = ns)) +
        drug.findall("{ns}products/{ns}product/{ns}name".format(ns = ns))

    }
    aliases.add(row['name'])
    row['aliases'] = sorted(aliases)

    rows.append(row)

In [8]:
alias_dict = {row['drugbank_id']: row['aliases'] for row in rows}
with open('./Primary_target_databases/DrugBank_2022/aliases.json', 'w') as fp:
    json.dump(alias_dict, fp, indent=2, sort_keys=True)

In [9]:
def collapse_list_values(row):
    for key, value in row.items():
        if isinstance(value, list):
            row[key] = '|'.join(value)
    return row

rows = list(map(collapse_list_values, rows))

In [10]:
columns = ['drugbank_id', 'name', 'type', 'groups', 'atc_codes', 'categories', 'inchikey', 'inchi', 'description']
drugbank_df = pandas.DataFrame.from_dict(rows)[columns]
drugbank_df.head()

,drugbank_id,name,type,groups,atc_codes,categories,inchikey,inchi,description
0,DB00001,Lepirudin,biotech,approved,B01AE02,"Amino Acids, Peptides, and Proteins|Anticoagul...",None,None,Lepirudin is identical to natural hirudin exce...
1,DB00002,Cetuximab,biotech,approved,L01XC06,"Amino Acids, Peptides, and Proteins|Antibodies...",None,None,Cetuximab is a recombinant chimeric human/mous...
2,DB00003,Dornase alfa,biotech,approved,R05CB13,"Amino Acids, Peptides, and Proteins|Cough and ...",None,None,Dornase alfa is a biosynthetic form of human d...
3,DB00004,Denileukin diftitox,biotech,approved|investigational,L01XX29,"ADP Ribose Transferases|Amino Acids, Peptides,...",None,None,A recombinant DNA-derived cytotoxic protein co...
4,DB00005,Etanercept,biotech,approved|investigational,L04AB01,"Agents reducing cytokine levels|Amino Acids, P...",None,None,Dimeric fusion protein consisting of the extra...


In [11]:
drugbank_df.shape

(14594, 9)

In [12]:
drugbank_slim_df = drugbank_df[
    drugbank_df.groups.map(lambda x: 'approved' in x) &
    drugbank_df.inchi.map(lambda x: x is not None) &
    drugbank_df.type.map(lambda x: x == 'small molecule')
]
drugbank_slim_df.head()

,drugbank_id,name,type,groups,atc_codes,categories,inchikey,inchi,description
5,DB00006,Bivalirudin,small molecule,approved|investigational,B01AE06,"Amino Acids, Peptides, and Proteins|Anticoagul...",OIRCOABEOLEUMC-GEJPAHFPSA-N,InChI=1S/C98H138N24O33/c1-5-52(4)82(96(153)122...,Bivalirudin is a synthetic 20 residue peptide ...
6,DB00007,Leuprolide,small molecule,approved|investigational,L02AE51|L02AE02,Adrenal Cortex Hormones|Agents Causing Muscle ...,GFIJNRVAKGFPGQ-LIJARHBVSA-N,InChI=1S/C59H84N16O12/c1-6-63-57(86)48-14-10-2...,Leuprolide is a synthetic 9-residue peptide an...
13,DB00014,Goserelin,small molecule,approved,L02AE03,"Adrenal Cortex Hormones|Amino Acids, Peptides,...",BLCLNMBMMGCOAS-URPVMXJPSA-N,InChI=1S/C59H84N18O14/c1-31(2)22-40(49(82)68-3...,"Goserelin is a synthetic hormone. In men, it s..."
25,DB00027,Gramicidin D,small molecule,approved,R02AB30,"Amino Acids, Peptides, and Proteins|Anti-Bacte...",NDAYQJDHGXTBJL-MWWSRJDJSA-N,InChI=1S/C96H135N19O16/c1-50(2)36-71(105-79(11...,Gramcidin D is a heterogeneous mixture of thre...
33,DB00035,Desmopressin,small molecule,approved,H01BA02,"Agents that produce hypertension|Amino Acids, ...",NFLWUMRGJYTJIN-PNIOQBSNSA-N,InChI=1S/C46H64N14O12S2/c47-35(62)15-14-29-40(...,"Desmopressin (dDAVP), a synthetic analogue of ..."


In [13]:
drugbank_slim_df.shape

(2727, 9)

In [14]:
# write drugbank tsv
path = os.path.join('./Primary_target_databases/DrugBank_2022/', 'drugbank.tsv')
drugbank_df.to_csv(path, sep='\t', index=False)

# write slim drugbank tsv
path = os.path.join('./Primary_target_databases/DrugBank_2022/', 'drugbank-slim.tsv')
drugbank_slim_df.to_csv(path, sep='\t', index=False)

## Extract protein information

In [15]:
protein_rows = list()
for i, drug in enumerate(root):
    drugbank_id = drug.findtext(ns + "drugbank-id[@primary='true']")
    for category in ['target', 'enzyme', 'carrier', 'transporter']:
        proteins = drug.findall('{ns}{cat}s/{ns}{cat}'.format(ns=ns, cat=category))
        for protein in proteins:
            row = {'drugbank_id': drugbank_id, 'category': category}
            row['organism'] = protein.findtext('{}organism'.format(ns))
            row['known_action'] = protein.findtext('{}known-action'.format(ns))
            actions = protein.findall('{ns}actions/{ns}action'.format(ns=ns))
            row['actions'] = '|'.join(action.text for action in actions)
            uniprot_ids = [polypep.text for polypep in protein.findall(
                "{ns}polypeptide/{ns}external-identifiers/{ns}external-identifier[{ns}resource='UniProtKB']/{ns}identifier".format(ns=ns))]            
            if len(uniprot_ids) != 1:
                continue
            row['uniprot_id'] = uniprot_ids[0]
            ref_text = protein.findtext("{ns}references[@format='textile']".format(ns=ns))
            if str(type(ref_text)) != "<class 'NoneType'>" :
                pmids = re.findall(r'pubmed/([0-9]+)', ref_text)
                row['pubmed_ids'] = '|'.join(pmids)
            else:
                row['pubmed_ids'] = None
            protein_rows.append(row)

protein_df = pandas.DataFrame.from_dict(protein_rows)

In [16]:
# Read our uniprot to entrez_gene mapping
response = requests.get('http://git.dhimmel.com/uniprot/data/map/GeneID.tsv.gz', stream=True)
text = io.TextIOWrapper(gzip.GzipFile(fileobj=response.raw))
uniprot_df = pandas.read_table(text, engine='python')
uniprot_df.rename(columns={'uniprot': 'uniprot_id', 'GeneID': 'entrez_gene_id'}, inplace=True)

# merge uniprot mapping with protein_df
entrez_df = protein_df.merge(uniprot_df, how='inner')

In [17]:
columns = ['drugbank_id', 'category', 'uniprot_id', 'entrez_gene_id', 'organism',
           'known_action', 'actions', 'pubmed_ids']
entrez_df = entrez_df[columns]

In [18]:
path = os.path.join('./Primary_target_databases/DrugBank_2022/', 'proteins.tsv')
entrez_df.to_csv(path, sep='\t', index=False)

In [19]:
entrez_df

,drugbank_id,category,uniprot_id,entrez_gene_id,organism,known_action,actions,pubmed_ids
0,DB00001,target,P00734,2147,Humans,yes,inhibitor,None
1,DB00006,target,P00734,2147,Humans,yes,inhibitor,None
2,DB00025,enzyme,P00734,2147,Humans,unknown,activator,None
3,DB00055,target,P00734,2147,Humans,unknown,,None
4,DB00100,target,P00734,2147,Humans,unknown,,None
...,...,...,...,...,...,...,...,...
27573,DB16628,target,O96007,4338,Humans,yes,substrate,None
27574,DB16698,target,Q14242,6404,Humans,unknown,agonist,None
27575,DB16733,target,Q92952,3780,Humans,yes,positive allosteric modulator,None
27576,DB16733,target,Q9H2S1,3781,Humans,yes,positive allosteric modulator,None


In [20]:
entrez_df.actions.value_counts()

                                         10775
inhibitor                                 5336
substrate                                 4481
antagonist                                1554
agonist                                   1139
                                         ...  
substrate|cleavage                           1
agonist|inhibitor|downregulator              1
antagonist|agonist|negative modulator        1
modulator|product of                         1
inhibitor|binder|modulator                   1
Name: actions, Length: 125, dtype: int64

In [21]:
# Number of unique genes with an interaction
len(set(entrez_df.entrez_gene_id))

4611

In [22]:
# Number of unique drugs  with an interaction
len(set(entrez_df.drugbank_id))

7173

In [23]:
drugbank_df.head()

,drugbank_id,name,type,groups,atc_codes,categories,inchikey,inchi,description
0,DB00001,Lepirudin,biotech,approved,B01AE02,"Amino Acids, Peptides, and Proteins|Anticoagul...",None,None,Lepirudin is identical to natural hirudin exce...
1,DB00002,Cetuximab,biotech,approved,L01XC06,"Amino Acids, Peptides, and Proteins|Antibodies...",None,None,Cetuximab is a recombinant chimeric human/mous...
2,DB00003,Dornase alfa,biotech,approved,R05CB13,"Amino Acids, Peptides, and Proteins|Cough and ...",None,None,Dornase alfa is a biosynthetic form of human d...
3,DB00004,Denileukin diftitox,biotech,approved|investigational,L01XX29,"ADP Ribose Transferases|Amino Acids, Peptides,...",None,None,A recombinant DNA-derived cytotoxic protein co...
4,DB00005,Etanercept,biotech,approved|investigational,L04AB01,"Agents reducing cytokine levels|Amino Acids, P...",None,None,Dimeric fusion protein consisting of the extra...


In [24]:
drugbank_df.columns

Index(['drugbank_id', 'name', 'type', 'groups', 'atc_codes', 'categories',
       'inchikey', 'inchi', 'description'],
      dtype='object')

In [25]:
# Maping multiple columns
drugbank_df.index = drugbank_df.drugbank_id
entrez_df.index = entrez_df.drugbank_id
for i in ['name', 'type', 'groups', 'atc_codes', 'categories', 'description']:
    entrez_df[i] = entrez_df.drugbank_id.map(drugbank_df[i])

In [26]:
entrez_df.head()

,drugbank_id,category,uniprot_id,entrez_gene_id,organism,known_action,actions,pubmed_ids,name,type,groups,atc_codes,categories,description
drugbank_id,,,,,,,,,,,,,,
DB00001,DB00001,target,P00734,2147,Humans,yes,inhibitor,None,Lepirudin,biotech,approved,B01AE02,"Amino Acids, Peptides, and Proteins|Anticoagul...",Lepirudin is identical to natural hirudin exce...
DB00006,DB00006,target,P00734,2147,Humans,yes,inhibitor,None,Bivalirudin,small molecule,approved|investigational,B01AE06,"Amino Acids, Peptides, and Proteins|Anticoagul...",Bivalirudin is a synthetic 20 residue peptide ...
DB00025,DB00025,enzyme,P00734,2147,Humans,unknown,activator,None,"Antihemophilic factor, human recombinant",biotech,approved|investigational,B02BD02,"Amino Acids, Peptides, and Proteins|Biological...",Human recombinant antihemophilic factor (AHF) ...
DB00055,DB00055,target,P00734,2147,Humans,unknown,,None,Drotrecogin alfa,biotech,approved|investigational|withdrawn,B01AD10,"Amino Acids, Peptides, and Proteins|Anti-Infec...",Drotrecogin alfa is activated human protein C ...
DB00100,DB00100,target,P00734,2147,Humans,unknown,,None,Coagulation Factor IX (Recombinant),biotech,approved|investigational,,Blood Coagulation Factors|Hemostatics|Increase...,Recombinant Coagulation Factor IX is a purifie...


In [27]:
# Mapping Entrez to gene symbol
glist = np.unique(entrez_df.entrez_gene_id.values).tolist()
glist = [str(x) for x in glist if type(x) ==int]
results = bm.query(dataset='hsapiens_gene_ensembl', attributes=['entrezgene_id','hgnc_symbol'],#, 'go_id'],
                   filters={'entrezgene_id': glist})

In [28]:
entrez_df

,drugbank_id,category,uniprot_id,entrez_gene_id,organism,known_action,actions,pubmed_ids,name,type,groups,atc_codes,categories,description
drugbank_id,,,,,,,,,,,,,,
DB00001,DB00001,target,P00734,2147,Humans,yes,inhibitor,None,Lepirudin,biotech,approved,B01AE02,"Amino Acids, Peptides, and Proteins|Anticoagul...",Lepirudin is identical to natural hirudin exce...
DB00006,DB00006,target,P00734,2147,Humans,yes,inhibitor,None,Bivalirudin,small molecule,approved|investigational,B01AE06,"Amino Acids, Peptides, and Proteins|Anticoagul...",Bivalirudin is a synthetic 20 residue peptide ...
DB00025,DB00025,enzyme,P00734,2147,Humans,unknown,activator,None,"Antihemophilic factor, human recombinant",biotech,approved|investigational,B02BD02,"Amino Acids, Peptides, and Proteins|Biological...",Human recombinant antihemophilic factor (AHF) ...
DB00055,DB00055,target,P00734,2147,Humans,unknown,,None,Drotrecogin alfa,biotech,approved|investigational|withdrawn,B01AD10,"Amino Acids, Peptides, and Proteins|Anti-Infec...",Drotrecogin alfa is activated human protein C ...
DB00100,DB00100,target,P00734,2147,Humans,unknown,,None,Coagulation Factor IX (Recombinant),biotech,approved|investigational,,Blood Coagulation Factors|Hemostatics|Increase...,Recombinant Coagulation Factor IX is a purifie...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DB16628,DB16628,target,O96007,4338,Humans,yes,substrate,None,Fosdenopterin,small molecule,approved,,Biological Factors|Cyclic Pyranopterin Monopho...,Molybdenum cofactor deficiency (MoCD) is an ex...
DB16698,DB16698,target,Q14242,6404,Humans,unknown,agonist,None,Neihulizumab,biotech,investigational,,,Neihulizumab is a novel immune checkpoint agon...
DB16733,DB16733,target,Q92952,3780,Humans,yes,positive allosteric modulator,None,Rimtuzalcap,small molecule,investigational,,,Rimtuzalcap is a novel modulator of small-cond...


In [29]:
entrez_df.index = np.arange(entrez_df.shape[0])
results =  results.dropna().drop_duplicates().reset_index(drop=True)
#entrez_df['hgnc_symbol'] = 0
entrez_df['hgnc_symbol'] = entrez_df.entrez_gene_id.map(results['hgnc_symbol'])
#for i in range(entrez_df.shape[0]):
#    entrez_df.iloc[i,-1] = results.loc[results.entrezgene_id==entrez_df.iloc[i,3],'hgnc_symbol'][0].values#entrez_df.entrez_gene_id.map(results['hgnc_symbol'])

In [30]:
results.shape

(2828, 2)

In [31]:
entrez_df.head()

,drugbank_id,category,uniprot_id,entrez_gene_id,organism,known_action,actions,pubmed_ids,name,type,groups,atc_codes,categories,description,hgnc_symbol
0,DB00001,target,P00734,2147,Humans,yes,inhibitor,None,Lepirudin,biotech,approved,B01AE02,"Amino Acids, Peptides, and Proteins|Anticoagul...",Lepirudin is identical to natural hirudin exce...,SFTPD
1,DB00006,target,P00734,2147,Humans,yes,inhibitor,None,Bivalirudin,small molecule,approved|investigational,B01AE06,"Amino Acids, Peptides, and Proteins|Anticoagul...",Bivalirudin is a synthetic 20 residue peptide ...,SFTPD
2,DB00025,enzyme,P00734,2147,Humans,unknown,activator,None,"Antihemophilic factor, human recombinant",biotech,approved|investigational,B02BD02,"Amino Acids, Peptides, and Proteins|Biological...",Human recombinant antihemophilic factor (AHF) ...,SFTPD
3,DB00055,target,P00734,2147,Humans,unknown,,None,Drotrecogin alfa,biotech,approved|investigational|withdrawn,B01AD10,"Amino Acids, Peptides, and Proteins|Anti-Infec...",Drotrecogin alfa is activated human protein C ...,SFTPD
4,DB00100,target,P00734,2147,Humans,unknown,,None,Coagulation Factor IX (Recombinant),biotech,approved|investigational,,Blood Coagulation Factors|Hemostatics|Increase...,Recombinant Coagulation Factor IX is a purifie...,SFTPD


In [32]:
entrez_df.to_csv('./Primary_target_databases/DrugBank_2022/DrugBank_Drug_Target.csv')